In [1]:
from groq import Groq
import json
import pandas as pd    
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from math import sqrt
import tqdm
import os
import numpy as np
import json
import concurrent.futures
import numpy as np
from tqdm import tqdm
import re
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.metrics import mean_squared_error


In [42]:
#API keys from groq
client = Groq(api_key="KEY1") 
api_keys=["KEY2","KEY3"] #get multiple API keys with different Email addresses, you will run into rate limits a lot so either slow down the requests or let lots of api keys handle the job
api_keyover=0

In [37]:
# Load the CSV file
csv_file = 'emails2.csv' 
df = pd.read_csv(csv_file)
print(len(df))

3225


In [35]:
# Load the dataset from Hugging Face
from datasets import load_dataset
import pandas as pd


dataset = load_dataset("gabrielmbmb/FinePersonas-Synthetic-Emails")
df = pd.DataFrame(dataset['train'])
df = df[df['raw_emails'].notna() & df['raw_emails'].str.strip().astype(bool)]


def split_emails(row):
    return [email.strip() for email in row.split('</email>') if email.strip()]

df['body'] = df['raw_emails'].apply(split_emails)
df_split = df.explode('body').reset_index(drop=True)

df_split = df_split.drop(columns=['raw_emails'])

df_split = df_split[['body']]
df_split


,body
0,"<email>\nFrom: Emily Johnson <ejohnson@oakwoodhigh.edu>\nTo: Michael Thompson <mthompson@oakwoodhigh.edu>\nSubject: Collaboration on Climate Change and Urbanization Project\n\nHey Michael,\n\nI hope you're doing well! I'm reaching out because I'm currently working on a lesson plan about the human impacts of climate change. As I was brainstorming ideas, I thought about your expertise in human geography and urban studies. I was wondering if you'd be interested in collaborating on a cross-curricular project for our students?\n\nI think it would be really valuable for our students to explore the relationship between urbanization and climate change. We could have them work together on a project that examines this topic and present their findings to the school and local community.\n\nLet me know what you think! I'd love to brainstorm some ideas together.\n\nBest,\nEmily"
1,"<email>\nFrom: Michael Thompson <mthompson@oakwoodhigh.edu>\nTo: Emily Johnson <ejohnson@oakwoodhigh.edu>\nSubject: RE: Collaboration on Climate Change and Urbanization Project\n\nEmily,\n\nWhat a fantastic idea! I'm so glad you reached out. I've actually been working on a similar lesson plan that focuses on the impact of urbanization on the environment. Combining our efforts and creating a cross-curricular project sounds like a great opportunity for our students.\n\nI think having the students present their findings to the school and local community is a brilliant way to engage them and make the project more meaningful. It could even potentially inspire some real change in our community!\n\nI'm excited to start brainstorming ideas for the project. Do you want to meet up sometime this week to discuss the details? Let me know when works best for you.\n\nBest,\nMichael"
2,"<email>\nFrom: Emily Johnson <ejohnson@oakwoodhigh.edu>\nTo: Michael Thompson <mthompson@oakwoodhigh.edu>\nSubject: RE: Collaboration on Climate Change and Urbanization Project\n\nMichael,\n\nI'm thrilled that you're on board with this project! It's so great to have a colleague like you to collaborate with. I think our students are going to get a lot out of this experience.\n\nI'm free to meet up on Wednesday afternoon if that works for you. We could grab a coffee and brainstorm some ideas for the project. I'm thinking we could have the students work in small groups and each group could focus on a different aspect of the topic. They could then present their findings to the class and the community.\n\nI'm also thinking we could potentially present this project at the upcoming National Geography Teachers' Conference. It could be a great opportunity to share our ideas with other educators and get some feedback.\n\nLet me know what you think about all of this! I'm looking forward to our meeting on Wednesday.\n\nBest,\nEmily"
3,"<email>\nFrom: Emily Thompson <emily.thompson@oakwoodhigh.edu>\nTo: Michael Rodriguez <michael.rodriguez@oakwoodhigh.edu>\nSubject: Need some advice on teaching climate change impacts\n\nHey Michael,\n\nI hope you're having a great week so far. I'm currently working on a new lesson plan for my environmental studies class, focusing on the human impacts of climate change. I know this is a complex and sensitive topic, and I want to make sure I approach it in the right way.\n\nI was wondering if you have any experience teaching this subject or if you have any resources that you could share with me. I would really appreciate any insights or suggestions you might have.\n\nLet me know if you have some time to chat about this, maybe over lunch one day next week?\n\nBest regards,\nEmily"
4,"<email>\nFrom: Michael Rodriguez <michael.rodriguez@oakwoodhigh.edu>\nTo: Emily Thompson <emily.thompson@oakwoodhigh.edu>\nSubject: RE: Need some advice on teaching climate change impacts\n\nHi Emily,\n\nIt's great to hear from you! I'm glad to know that you're tackling such an important topic in your class. Teaching about the human impacts of climate change can be challenging, but it's cru

In [36]:
# Save the updated DataFrame back to a new CSV file
output_file = 'emails2.csv'
df_split.to_csv(output_file, index=False)

In [40]:
if 'body' in df.columns:
    # prompt cannot be too long just a check to limit incase an email is too large
    max_length = 1000000000
    short_emails = df[df['body'].str.len() <= max_length]
    
else:
    print("The 'Body' column does not exist in the CSV file.")


In [44]:
model='llama3-8b-8192'
# model = "llama3-70b-8192"

def get_prediction(df, model=model):
    global i, api_keyover, client
    
    # List to hold the results
    results = []

    for email_body in df['body']: 
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=[
                    {
                        "role": "system",
                        "content": """You are a helpful assistant that classifies emails into three categories and determines if they are sensitive."""
                    },
                    {
                        "role": "user",
                        "content": f"""TCheck if this email is related to a college Head of Department (HOD). 
                                        Categorize it as one of the following categories: [student, research, corporate,other]. 

                                        Keywords to help you:
                                        - For 'student': course, admission, syllabus, grades, exam, registration.
                                        - For 'research': collaboration, joint project, data sharing, research paper, study.
                                        - For 'corporate': business, partnership, contract, legal, sponsorship.

                                        Additionally, determine if the email is sensitive or confidential. If it mentions sensitive topics like legal matters, corporate partnerships, or confidential information, mark it as "Yes" for sensitivity. Otherwise, mark it as "No".

                                        Just return:
                                        [student, research,corporate or other],[Yes or No] WITHOUT BRACKET
                                        
                                        Email body:
                                        {email_body}
                                        """
                    }
                ],
                temperature=1,
                top_p=1,
                stream=False,
                stop=None,
            )
        except Exception as e:
            print('API key limit reached', e)
            api_keyover += 1
            client = Groq(api_key=api_keys[api_keyover % 2])
            continue

        try:
            answer = completion.choices[0].message.content.strip()
            category, sensitivity = answer.split(',')
            if (category == "other"):
                continue
            results.append((email_body, category.strip(), sensitivity.strip()))
        except Exception as e:
            continue

    result_df = pd.DataFrame(results, columns=['body', 'label', 'sensitivity'])
    
    return result_df


In [50]:
# Function to process dataframe enteries in batches
def process_in_batches(df, batch_size):
    total_rows = len(df)  
    final_df = pd.DataFrame()  


    for start in range(0, total_rows, batch_size):
        end = min(start + batch_size, total_rows) 
        batch_df = df.iloc[start:end] 


        result_df = get_prediction(batch_df,model)  

      
        final_df = pd.concat([final_df, result_df], ignore_index=True)
        print(f"Done labeling {end} entries")

    return final_df  


In [47]:
#Processing Chunks of data (200enteries per chunk)

start_index = 0
end_index = 200 
step_size = 200  
batch_size = 10  


while end_index <= len(short_emails):
    
    current_batch = short_emails[start_index:end_index]
    final_results_df = process_in_batches(current_batch, batch_size=batch_size)

    output_file = f'data_10{start_index // step_size+1}.csv'  
    final_results_df.to_csv(output_file, index=False)

    print(f"Processed and saved emails from index {start_index} to {end_index} as {output_file}")

    start_index = end_index
    end_index += step_size

Done labeling 10 entries
Done labeling 20 entries
Done labeling 30 entries
Done labeling 40 entries
Done labeling 50 entries
API key limit reached Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j0p6w2wkf66sgphs6ncrb8vd` on : Limit 500000, Used 499858, Requested 514. Please try again in 1m4.163399999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
API key limit reached Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j0p6w2wkf66sgphs6ncrb8vd` on : Limit 500000, Used 499854, Requested 566. Please try again in 1m12.53s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
API key limit reached Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j0p6w2wkf66sgphs6ncrb8vd` on : Limi

In [22]:
# Coverting some fixed files to desired format [All students or all research]


csv_file = 'Synthetic_students - Sheet1.csv'  
df = pd.read_csv(csv_file)

df['label'] = 'student'  
df['sensitivity'] = 'No'          

# Save the updated DataFrame back to a new CSV file
output_file = 'student_mails2.csv'
df.to_csv(output_file, index=False)



In [23]:
# Load the CSV file to check it once

csv_file = 'Labeled_data/student_mails2.csv'  
df = pd.read_csv(csv_file)
print(len(df))
df

109


,body,label,sensitivity
0,"Hi Professor,\n I'm writing to clarify the req...",student,No
1,"Dear Professor,\n I'm having trouble grasping ...",student,No
2,"Hello Professor Sameer,\n I'm working on my Re...",student,No
3,"Dear Professor Sameer,\n I'm conducting a surv...",student,No
4,"Dear Professor Patel, I am a fourth-year stude...",student,No
...,...,...,...
104,"Dear Professor Nair, I was unable to participa...",student,No
105,"Hi Dr. Patel, I missed the Microeconomics exam...",student,No
106,"Hello Professor Rao, I wasn't able to take the...",student,No
107,"Dear Ms. Verma, I could not attend the Microec...",student,No


In [24]:
# Cocatanating data from all CSV files

directory = 'Labeled_data'

all_data = pd.DataFrame(columns=['body', 'label', 'sensitivity'])

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        if 'body' in df.columns and 'label' in df.columns and 'sensitivity' in df.columns:
            selected_df = df[['body', 'label', 'sensitivity']]
        else:
            raise ValueError(f"Columns missing in file {filename}")

        all_data = pd.concat([all_data, selected_df], ignore_index=True)

all_data

,body,label,sensitivity
0,Message-ID: <18782981.1075855378110.JavaMail.e...,corporate,No
1,Message-ID: <15464986.1075855378456.JavaMail.e...,corporate,No
2,Message-ID: <24216240.1075855687451.JavaMail.e...,student,No
3,Message-ID: <13505866.1075863688222.JavaMail.e...,corporate,Yes
4,Message-ID: <16254169.1075863688286.JavaMail.e...,research,No
...,...,...,...
5511,"Dear Professor Nair, I was unable to participa...",student,No
5512,"Hi Dr. Patel, I missed the Microeconomics exam...",student,No
5513,"Hello Professor Rao, I wasn't able to take the...",student,No
5514,"Dear Ms. Verma, I could not attend the Microec...",student,No


In [29]:
all_data
all_data[all_data['label']=="corporate"]

,body,label,sensitivity
0,Message-ID: <18782981.1075855378110.JavaMail.e...,corporate,No
1,Message-ID: <15464986.1075855378456.JavaMail.e...,corporate,No
3,Message-ID: <13505866.1075863688222.JavaMail.e...,corporate,Yes
8,Message-ID: <33076797.1075855687515.JavaMail.e...,corporate,No
13,Message-ID: <1115198.1075855687626.JavaMail.ev...,corporate,No
...,...,...,...
1919,Message-ID: <24640754.1075855677306.JavaMail.e...,corporate,Yes
1921,Message-ID: <17966419.1075855677350.JavaMail.e...,corporate,No
1922,Message-ID: <5272543.1075855677371.JavaMail.ev...,corporate,No
1923,Message-ID: <31944468.1075855677393.JavaMail.e...,corporate,No


In [28]:
student_rows = all_data[all_data['label'] == 'student']
corporate_rows = all_data[all_data['label'] == 'corporate']

research_rows = all_data[all_data['label'] == 'research'].sample(n=1087, random_state=1)

other_rows = all_data[all_data['label'] == 'Other'].sample(n=40, random_state=1)


final_df = pd.concat([student_rows, corporate_rows, research_rows, other_rows], ignore_index=True)

final_df


,body,label,sensitivity
0,Message-ID: <24216240.1075855687451.JavaMail.e...,student,No
1,Message-ID: <25459584.1075855687536.JavaMail.e...,student,No
2,Message-ID: <13116875.1075855687561.JavaMail.e...,student,No
3,Message-ID: <2707340.1075855687584.JavaMail.ev...,student,No
4,Message-ID: <24729148.1075855687713.JavaMail.e...,student,No
...,...,...,...
2995,Message-ID: <25865671.1075857651354.JavaMail.e...,Other,No
2996,Message-ID: <26848912.1075855715451.JavaMail.e...,Other,Yes
2997,Message-ID: <1344885.1075857600283.JavaMail.ev...,Other,No
2998,Message-ID: <27039998.1075857599657.JavaMail.e...,Other,No


In [40]:


label_counts = final_df['label'].value_counts()
print("Label counts in the DataFrame:")
print(label_counts)


train_size = 2500
test_size = 500

# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(final_df, train_size=2500, test_size=500, random_state=42)



Label counts in the DataFrame:
label
corporate    1247
research     1087
student       626
Other          40
Name: count, dtype: int64


In [5]:
# Display the distribution of labels in the train and test sets
train_distribution = train_df['label'].value_counts(normalize=True)
test_distribution = test_df['label'].value_counts(normalize=True)

print("\nTraining Set Label Distribution:")
print(train_distribution)

print("\nTesting Set Label Distribution:")
print(test_distribution)


In [6]:
train_df = pd.read_csv("Data/train.csv")
test_df = pd.read_csv("Data/test.csv")

In [15]:
#Cleaning

# Function to clean the email body
def clean_email_body(email_body):
    if isinstance(email_body, str):
        clean_body = re.sub(r"(?s)(Message-ID:.*?X-FileName:.*?\n|From:.*?\n|To:.*?\n|Subject:.*?\n|FORWARDED.*?\n|cc:.*?)", "", email_body)
        return clean_body.strip()
    else:
        return email_body  


train_df['body'] = train_df['body'].apply(clean_email_body)
test_df['body'] = test_df['body'].apply(clean_email_body)


In [16]:
output_file = "Data/train.csv"
train_df.to_csv(output_file,index=True)

output_file = "Data/test.csv"
test_df.to_csv(output_file,index=True)